In [23]:
import regex as re
import glob
import pandas as pd

items = {}

def clean(s):
    return re.sub('[ \t\n]+', ' ', re.sub(r'[\<\/\>]', ' ', s)).strip()

def find_entities(s):
    n = 0
    stack = []
    tag_locs = []
    entityList = []
    for i, c in enumerate(s):
        if s[i] == '<':
            stack.append(i)
            print("stack at " + c + ": " + '[%s]' % ', '.join(map(str, stack)))
        elif s[i] == '/' and s[i+1] == '>':
            try:
                tag_locs.append((stack.pop(), i))
                print("locs at " + c + ": " + '[%s]' % ', '.join(map(str, tag_locs)))
            except IndexError:
                raise IndexError('Too many closing tags at index {} in {}'.format(i,s))
        if not stack and tag_locs:
            entity_loc = tag_locs.pop() 
            entity = clean(s[entity_loc[0]:entity_loc[1]])
            entityList.append(({entity : [entity] + [clean(s[loc[0]:loc[1]]) for loc in tag_locs]}))
            tag_locs = []
            print("entity list: " + '[%s]' % ', '.join(map(str, entityList)))
    if stack:
        print("stack: " + '[%s]' % ', '.join(map(str, stack)))
        raise IndexError('Unbalanced tags at index {} in {}'.format(stack.pop(),s))
    return entityList

reviews = [open(filename, "r").read() for filename in glob.glob("dataset/*.txt")]
items = {review: find_entities(review) for review in reviews}

items

stack at <: [236]
stack at <: [236, 237]
locs at /: [(237, 244)]
stack at <: [236, 247]
locs at /: [(237, 244), (247, 253)]
locs at /: [(237, 244), (247, 253), (236, 255)]
entity list: [{'kimchi fries': ['kimchi fries', 'kimchi', 'fries']}]
stack at <: [284]
stack at <: [284, 288]
locs at /: [(288, 294)]
locs at /: [(288, 294), (284, 296)]
entity list: [{'kimchi fries': ['kimchi fries', 'kimchi', 'fries']}, {'NY Fries': ['NY Fries', 'Fries']}]
stack at <: [316]
stack at <: [316, 325]
stack at <: [316, 325, 326]
locs at /: [(326, 331)]
stack at <: [316, 325, 334]
locs at /: [(326, 331), (334, 342)]
locs at /: [(326, 331), (334, 342), (325, 344)]
locs at /: [(326, 331), (334, 342), (325, 344), (316, 346)]
entity list: [{'kimchi fries': ['kimchi fries', 'kimchi', 'fries']}, {'NY Fries': ['NY Fries', 'Fries']}, {'grilled pork banh mi': ['grilled pork banh mi', 'pork', 'banh mi', 'pork banh mi']}]
stack at <: [401]
locs at /: [(401, 405)]
entity list: [{'kimchi fries': ['kimchi fries', 'kim

{'<burgers/> are very big portions here.\n\ndefinitely order the <<onion/> ring/> tower to share...\n\n<Milkshakes/> are tasty! My personal favourite - the <vanilla/> one.': [{'burgers': ['burgers']},
  {'onion ring': ['onion ring', 'onion']},
  {'Milkshakes': ['Milkshakes']},
  {'vanilla': ['vanilla']}],
 "<hand made <noodles/>/>. fast. great service. open kitchen. Need i to say more?\n\nThere was a line up when we got there, but it moves pretty fast (approximately 10 mins wait). As soon as the server sees you there waiting, he offers you a glass of water.\n\nStaff is extremely friendly and nice. Service is really fast. Even faster when you speak chinese to them.\n\nMy boyfriend and I had ordered the <<soup/> <dumplings/>/>, <sweet and sour <<veggie/> <<pork/> <noodle/>/>/>/> and a <<beef/> <noodle/>/>. The <<soup/> <dumplings/>/> were good, as they are made on the spot. The <<veggie/> <<pork/> <handmade <noodle/>/>/>/> was really good. The <<beef/> <noodle/>/>, not so much.\n\nThey a

In [27]:
import json
with open('result.json', 'w') as fp:
    json.dump(items, fp)